In [85]:
from neo4j import GraphDatabase, basic_auth
import os

driver = GraphDatabase.driver("bolt://localhost:7687",auth=basic_auth("neo4j", "Kaffe1234"))

In [86]:
def get_count_of_type(label, session):
    q = "MATCH (n:%s) RETURN count(n)" % label
    res = session.run(q)
    print("Added", res.value()[0], "nodes of type", label)
    
def get_count_of_relationship(rel_label, fromLabel, toLabel, session):
    q = "MATCH (:%s)-[r:%s]-(:%s) RETURN count(*)" % (fromLabel, rel_label, toLabel)
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", fromLabel, rel_label, toLabel)

def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("Data/csvs/basal_ganglia/sources")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

In [106]:
# Deleting all source data in database
with driver.session() as session:
        session.run('''
            MATCH (n)
            WHERE  n:Publication
                OR n:Journal 
                OR n:ExclusionReason 
                OR n:Repository 
                OR n:Source
                OR n:RawData
                OR n:ConsideredPaper
                OR n:ExclusionReason
            DETACH DELETE n
        ''')
        print("wipeddatabase")

wipeddatabase


In [107]:
# Adding Publication to graph from sources.csv

csv_file_path = "file:///%s" % get_csv_path("sources.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        WITH row WHERE row.Source_type = 'Publication'
        MERGE (:Publication {id: row.ID, title: row.Source_title, insertedData: row.Inserted_date, publicationYear: toInt(row.Source_publication_year), sourceName: row.Source_name})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Publication", session)

Added 239 nodes of type Publication


In [108]:
# Adding Repository to graph from sources.csv

csv_file_path = "file:///%s" % get_csv_path("sources.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        WITH row WHERE row.Source_type = 'Repository'
        MERGE (:Repository {id: row.ID, title: row.Source_title, insertedData: row.Inserted_date, yearPublished: toInt(row.Source_publication_year), sourceName: row.Source_name})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Repository", session)

Added 6 nodes of type Repository


In [109]:
# Adding Journal to graph from sources.csv

csv_file_path = "file:///%s" % get_csv_path("source_origins_lookup.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        WITH row WHERE row.ID <> '1' AND row.ID <> '36' AND row.ID <> '37'
        CREATE (:Journal {id: row.ID, name: row.Source_name, identifier: row.Identifier})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Journal", session)

Added 360 nodes of type Journal


In [110]:
# Adding Source to graph from sources.csv

csv_file_path = "file:///%s" % get_csv_path("repositories.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Source {id: row.ID, name: row.Source_name, identifier: row.Identifier})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Source", session)

Added 3 nodes of type Source


In [111]:
# Adding ConsideredPaper to graph from sources.csv
## TODO try to connect considered_paper and publication 
## based on first auth + year (published) with Source_name

csv_file_path = "file:///%s" % get_csv_path("considered_papers.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:ConsideredPaper {id: row.ID, title: row.Title, publishedYear: toInt(row.Published), firstAuthor: row.First_author, isIncluded: false})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    session.run("create index on :ConsideredPaper(id)")
    get_count_of_type("ConsideredPaper", session)
    
csv_file_path = "file:///%s" % get_csv_path("considered_papers_desicions.csv")

included_query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        WITH row WHERE row.Decision =~ ".*(?i)Included.*"
        MATCH (n:ConsideredPaper {id: row.Paper})
        SET n.isIncluded = true
        RETURN COUNT(n)
    """ % csv_file_path

with driver.session() as session:    
    res = session.run(included_query)
    print("Number of considered papers that are included:",  res.value()[0])

Added 2204 nodes of type ConsideredPaper
Number of considered papers that are included: 244


In [112]:
### Process considered paper decisions to find reasons where considered paper can get a relation to the reaseon.
## IF it doesnt have relation to excluded it  is included 

import pandas as pd
csv_file_path = "file:///%s" % get_csv_path("considered_papers_desicions.csv")
df = pd.read_csv(csv_file_path)

exclusion_reasons = df["Decision"].unique()

with driver.session() as session: 
    for reason in exclusion_reasons:
        if type(reason) == str and "included" not in reason.lower():

            q = "CREATE (:ExclusionReason {reason: '%s'})" % reason.replace("Excluded: ", "")
            session.run(q)
            
            # Relationship EXCLUSION_REASON between ConsideredPaper and ExclusionReason
            query="""
                    LOAD CSV WITH HEADERS FROM "%s" AS row
                    WITH row where row.Decision = '%s'
                    MATCH (a:ConsideredPaper { id: row.Paper})
                    MATCH (c:ExclusionReason { reason: '%s' })
                    MERGE (a)-[:EXCLUSION_REASON]->(c)
                """ % (csv_file_path, reason,  reason.replace("Excluded: ", ""))

            session.run(query)
    get_count_of_type("ExclusionReason", session)
    get_count_of_relationship("EXCLUSION_REASON", "ConsideredPaper", "ExclusionReason", session)



Added 13 nodes of type ExclusionReason
Added 2113 relationships of type ConsideredPaper EXCLUSION_REASON ExclusionReason


In [113]:
# Relationship PUBLISHED_IN between Journal and Publication
csv_file_path = "file:///%s" % get_csv_path("sources.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Publication { id: row.ID})
        MATCH (c:Journal { id: row.Source_origin })
        MERGE (a)-[:PUBLISHED_IN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PUBLISHED_IN", "Journal", "Publication", session)

Added 239 relationships of type Journal PUBLISHED_IN Publication


In [114]:
# Relationship PUBLISHED_IN between Journal and Publication
csv_file_path = "file:///%s" % get_csv_path("sources.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Repository { id: row.ID})
        MATCH (c:Source { id: row.Source_origin })
        MERGE (a)-[:RETRIEVED_FROM]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("RETRIEVED_FROM", "Repository", "Source", session)

Added 6 relationships of type Repository RETRIEVED_FROM Source
